# MetOncoFit Example
**Author:** Scott Campit

## Summary
This notebook generates the cancer tissue models discussed in Oruganty, K., Campit, S. E., Mamde, S., Lyssiotis, C. A., & Chandrasekaran, S. (2020). Common biochemical properties of metabolic genes recurrently dysregulated in tumors. Cancer & metabolism, 8(1), 1-15.

## 1. Load Data
The data for MetOncoFit is stored in `/lib`. 

In [55]:
import os
import zipfile

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn import preprocessing

We'll write up some accessory functions for data loading and processing.

In [47]:
def long_cancer_names(fileName: str) -> str:
    """Converts shorthand to longhand names for each tissue model."""
    canc = fileName.replace(".csv", "")
    canc_dict = {
        'breast': 'Breast Cancer',
        'cns': 'Glioma',
        'colon': 'Colorectal Cancer',
        'complex': 'Pan Cancer',
        'leukemia': 'B-cell lymphoma',
        'melanoma': 'Melanoma',
        'nsclc': 'Lung Cancer',
        'ovarian': 'Ovarian Cancer',
        'prostate': 'Prostate Cancer',
        'renal': 'Renal Cancer'
        }
    canc = canc_dict.get(canc)
    return canc

def long_feature_names(labelFileName: str) -> dict:
    """Converts shorthand to longhand names for each feature."""
    columnName_map = pd.read_csv(
        labelFileName, sep='\t', names=['Original', 'New'])
    bestNames = dict([(index, name)
                      for index, name in zip(columnName_map['Original'], columnName_map['New'])])
    return bestNames

def load_data(dataModelPath: str, labelFilePath: str) -> pd.DataFrame:
    """
    load_data reads in the cancer model data (.csv file) and outputs a pandas dataframe.

    :params:
        model_file: The path to the .csv file containing the rows as observations and the columns as features.
            Note: there needs to be a corresponding 'Genes' and 'Cell Line' column to set as the index.

    :return:
        model:      A pandas dataframe containing the cancer model data, with observations as rows and
            features as columns.
        cancer:     A string denoting the tissue type from the name of the .csv file.
    """

    column_names = long_feature_names(labelFilePath)
    if '.zip' in dataModelPath:
        zipfolder = os.path.dirname(data_path)
        filename = os.path.basename(data_path)
        cancer    = os.path.splitext(os.path.basename(data_path))[0]
        archive = zipfile.ZipFile(zipfolder, 'r')
        with archive.open(filename) as f:
            datamodel    = pd.read_csv(f)
            datamodel    = datamodel.rename(columns=column_names)
            datamodel    = datamodel.set_index(['Genes', 'Cell Line'])
    else:
        cancer       = dataModelPath.strip(".")[0]
        datamodel    = pd.read_csv(dataModelPath)
        datamodel    = datamodel.rename(columns=column_names)
        datamodel    = datamodel.set_index(['Genes', 'Cell Line'])

    return datamodel, cancer

Let's load the breast cancer model as an example. Note that you can replace the breast cancer model with the other tissue models. For tissues not discussed in the main manuscript, please access the Zenodo data repository DOI:10.5281/zenodo.3520696.

In [48]:
data_path = './../lib/data_models.zip/breast.csv'
labels = './../lib/headers.txt'
breast_model, breast_label = load_data(data_path, labels)

In [49]:
breast_model.head(3)

Flux change in Alanine and Aspartate Metabolism after gene KO  \
Genes Cell Line                                                                  
A4GNT BT-549                                              0.027766               
      HS-578-T                                            0.025870               
      MCF7                                                0.013893               

                 Flux change in Aminosugar Metabolism after gene KO  \
Genes Cell Line                                                       
A4GNT BT-549                                                   0.0    
      HS-578-T                                                 0.0    
      MCF7                                                     0.0    

                 Flux change in Arginine and Proline Metabolism after gene KO  \
Genes Cell Line                                                                 
A4GNT BT-549                                              1.274142              
      HS-578-T                                            1.274176              
      MCF7                                                1.910480              

                 Flux change in Bile Acid Biosynthesis after gene KO  \
Genes Cell Line                                                        
A4GNT BT-549                                                   0.0     
      HS-578-T                                                 0.0     
      MCF7                                                     0.0     

                 Flux change in Biotin Metabolism after gene KO  \
Genes Cell Line                                                   
A4GNT BT-549                                                0.0   
      HS-578-T                                              0.0   
      MCF7                                                  0.0   

                 Flux change in Cholesterol Metabolism after gene KO  \
Genes Cell Line                                                        
A4GNT BT-549                                              0.007901     
      HS-578-T                                            0.008405     
      MCF7                                                0.010212     

                 Flux change in Citric Acid Cycle after gene KO  \
Genes Cell Line                                                   
A4GNT BT-549                                           0.174623   
      HS-578-T                                         0.087882   
      MCF7                                             0.653827   

                 Flux change in CoA Biosynthesis after gene KO  \
Genes Cell Line                                                  
A4GNT BT-549                                               0.0   
      HS-578-T                                             0.0   
      MCF7                                                 0.0   

                 Flux change in CoA Catabolism after gene KO  \
Genes Cell Line                                                
A4GNT BT-549                                             0.0   
      HS-578-T                                           0.0   
      MCF7                                               0.0   

                 Flux change in Cysteine Metabolism after gene KO  ...  \
Genes Cell Line                                                    ...   
A4GNT BT-549                                             0.003353  ...   
      HS-578-T                                           0.002500  ...   
      MCF7                                               0.002500  ...   

                 Sum of topological distances to biomass components  \
Genes Cell Line                                                       
A4GNT BT-549                                                  3300    
      HS-578-T                                                3300    
      MCF7                                                    3300    

                 Catalytic efficiency  NCI-60 gene expression  \
Genes 

## 2. Data Preprocessing

### A. Label encoding

In [50]:
def label_encode(datamodel: pd.DataFrame) -> pd.DataFrame:
    """
    label_encode uses the label_encoder function from scikit-learn for the RECON1 subsystem and Metabolic subnetwork
        features. Note that these features may be removed in future MetOncoFit versions.

    :params:
        model:               A pandas dataframe containing the cancer model data, with observations as rows and
            features as columns.

    :return:
        label_encoded_model: A panda dataframe of the label-encoded model.
    """

    label_encoder = preprocessing.LabelEncoder()
    datamodel['RECON1 subsystem'] = label_encoder.fit_transform(
        datamodel['RECON1 subsystem'])
    datamodel['Metabolic subnetwork'] = label_encoder.fit_transform(
        datamodel['Metabolic subnetwork'])
    label_encoded_model = datamodel.copy(deep=True)

    return label_encoded_model

In [51]:
encoded_df = label_encode(breast_model)

### B. Split targets from data model
Let's separate the targets from the data model. In this example, we are trying to predict differentially expressed genes.

In [53]:
# Remove targets and features not of interest for differential expression prediction
encoded_df = encoded_df.drop(['TCGA gene expression fold change',
                              'CNV gain/loss ratio',
                              'CNV', 
                              'SURV'], axis=1)

# Separate out targets versus data model
target = encoded_df['TCGA annotation']
encoded_df = encoded_df.drop('TCGA annotation', axis=1)

### C. Robust Scaling
To account for the high dynamic range within the dataset, we will use robust scaling to scale the data.

In [56]:
from sklearn.preprocessing import RobustScaler

data = np.array(encoded_df).astype(np.float)
robust_df = RobustScaler(with_centering=True, with_scaling=True).fit_transform(data)

### D. Random oversampling
We also have an issue of class imbalance. To address this issue, we perform random oversampling. We'll first split the data using an 80/20 split.

In [58]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Split into training and test set
Xtrain, Xtest, Ytrain, Ytest = train_test_split(robust_df, target,
                                                test_size=0.2,
                                                train_size=0.8,
                                                shuffle=True)

# Random oversampling
over_sampler = RandomOverSampler(sampling_strategy='auto')
Xtrain, Ytrain = over_sampler.fit_resample(Xtrain, Ytrain)

## 3. Model Training
Now that we have the dataset preprocessed, it's time to train the random forest classifier.

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Train models
rfc = RandomForestClassifier()
rfc.fit(Xtrain, Ytrain)

# Evaluate
rfc_pred = rfc.predict(Xtest)
mean_acc = rfc.score(Xtest, Ytest)

Save the model.

In [63]:
import pickle

savepath = './../lib/breast_model.pkl'
with open(savepath, 'wb') as f:
    pickle.dump(rfc, f)

## 4. Evaluate random forest classifier model

In [62]:
feature_imp = pd.Series(rfc.feature_importances_,
                        index=encoded_df.columns).sort_values(ascending=False)
feature_imp.head(10)

NCI-60 gene expression                                          0.140468
Catalytic efficiency                                            0.056964
RECON1 subsystem                                                0.052971
Topological distance from reduced glutathione (media)           0.046240
Topological distance to CMP (biomass)                           0.043372
Topological distance from glutamine (media)                     0.041577
Sum of topological distances to media components                0.040358
Flux change in Arginine and Proline Metabolism after gene KO    0.034231
Topological distance to ATP (biomass)                           0.021931
Metabolic subnetwork                                            0.021653
dtype: float64